<h1><center>Einführung in die Neuroinformatik - Übungsblatt 4</center></h1>
<h2><center>Dominik Authaler, Marco Deuscher und Carolin Schindler</center></h2>
<h2><center>Gruppe Q</center></h2>
<h2><center>Mai 2019</center></h2>

### Aufgabe 1:

Von der Aufgabenstellung gegeben:
$w = \left(\begin{array}{c}1\\2\\\end{array}\right)$,    $w_0 = -1$,   $p = \left(\begin{array}{c}2\\2\\\end{array}\right) \in w_{-1}$  
  
1.) Bestimmung der aktuellen Lage der Trennunslinie:  
$$
\begin{align}
    w^Tx+w_0 = 0
    \Leftrightarrow \text{(1  2)}\left(\begin{array}{c}2\\2\\\end{array}\right) - 1 = 0
    \Leftrightarrow x_1 + 2x_2 - 1 = 0
    \Leftrightarrow x_2 = -\frac{1}{2}x_1 + \frac{1}{2}
\end{align}
$$
![title](b04_1.png)

2.)Erweiterung der Vektoren  
Damit die Erweiterung nichts ändert muss folgende Gleichung Äquivalenz gegebene sein:
$$
\begin{align}
    w^Tx+w_0 = 0
    \Leftrightarrow {w^*}^Tx^* = 0
\end{align}
$$
  
Wählt man $w^* = \left(\begin{array}{c}w_0\\w_1\\w_2\\\end{array}\right) = \left(\begin{array}{c}-1\\1\\2\\\end{array}\right)$, so ergibt sich für den erweiterten Datenpunkt
$p^* = \left(\begin{array}{c}1\\2\\2\\\end{array}\right)$.
  
3.) siehe Grafik bei 1.)
  
4.) Ausführen eines Lernschrittes:
Test, ob $p^*$ korrekt klassifiziert ist: ${w^*}^Tp^* = \text{(-1 1 2)}\left(\begin{array}{c}1\\2\\2\\\end{array}\right) = 5 > 0$. Da $p^*$ laut Vorgabe aber Teil von $w_{-1}$ ist, ist also eine Anpassung der Gewichte nötig:
  
$w^*(t+1) = w^*(t) - p^* = \left(\begin{array}{c}-1\\1\\2\\\end{array}\right) - \left(\begin{array}{c}1\\2\\2\\\end{array}\right) = \left(\begin{array}{c}-2\\-1\\0\\\end{array}\right)$
bzw. ausgedrückt ohne die erweiterten Vektoren:
$\tilde{w} = \left(\begin{array}{c}-1\\0\\\end{array}\right)$ und $\tilde{w_0} = -2$
![title](b04_2.png)

### Aufgabe 2:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

In [2]:
data = np.array([[-3, 1, -1], 
                 [-3, 3, 1], 
                 [-2, 1, -1], 
                 [-2, 4, 1], 
                 [-1, 3, 1], 
                 [-1, 4, 1], 
                 [2, 2, -1], 
                 [2, 4, 1], 
                 [3, 2, -1], 
                 [4, 1, -1]])
data.shape

(10, 3)

## Teil 1: Implementierung

In [3]:
weight_vector = np.zeros(3, dtype=np.float32)
weight_vectors = []
eta = 1
change = True

count = 0

while change and count < 100:
    change = False
    for dataPoint in data:
        #increase count to prevent endless loops
        count += 1
        
        target = dataPoint[2]
        
        #create new data vector with third coordinate
        newData = [1, dataPoint[0], dataPoint[1]]
        
        #calculate dot product
        res = np.dot(newData, weight_vector)
        
        if target > 0 and res < 0:
            weight_vector = weight_vector + eta * newData
            change = True
        elif target < 0 and res >= 0:
            weight_vector = weight_vector - eta * newData
            change = True
        
        weight_vectors.append(weight_vector)
        
        #print('dataPoint: ' + str(dataPoint) + '\tTarget: ' + str(target))
        #print('newData: ' + str(newData))
        #print('newWeight: ' + str(weight_vector) + '\n')

#print('Iterationen: ' + str(len(weight_vectors)))
    


## Teil 2: Darstellung

In [21]:
#TODO: Flächen in Legende markieren

import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import OrderedDict
from ipywidgets import *

def plotStep(step):
    #calculate data element used for this step
    cData = step % len(data)
    cPoint = data[cData]
    cWeight = weight_vectors[step]
    
    plt.figure(figsize=(7 ,7))
    plt.title('step = ' + str(step) + r', $w_0 = $' + str(cWeight[0]) + r', $w = ($' + str(cWeight[1]) + r', ' + str(cWeight[2]) + r')')
    plt.xlim(right=5,left=-5)
    plt.ylim(top=5,bottom=-5)
    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")
    plt.grid(color='lightgrey', linestyle='-', linewidth=1)
    plt.gca().set_axisbelow(True)
    plt.gca().xaxis.set_ticks(np.arange(-5, 6, 1))
    plt.gca().yaxis.set_ticks(np.arange(-5, 6, 1))

    x1 = np.arange(-5,5,0.01)
    
    for d in data:
        if d[2] == 1:
            color = 'green'
            label = r'$\omega_1$'
        else: 
            color = 'blue'
            label = r'$\omega_{-1}$'
            
        if (d == cPoint).all():
            plt.plot(cPoint[0],cPoint[1], marker="D",color=color,markersize=10, label='Processed point')
        else:
            plt.scatter(d[0], d[1], alpha=0.8, c=color, edgecolors='none', s=50, label=label)
    
    #draw the seperation line
    if(cWeight[2]!=0):
        x2 = -(cWeight[1] * x1 + cWeight[0]) / cWeight[2]
        plt.plot(x1,x2,label='Separation line')
        
        #TODO add handels to legend
        h1 = plt.fill_between(x1, x2, -5, color='blue', alpha='0.2')
        h2 = plt.fill_between(x1, x2, 5, color='green', alpha='0.2')
    else:
        #draw vertical line
        plt.axvline(x=0,label="Separation line")
        plt.axvspan(-5,0,-5,5,facecolor='blue', alpha=0.2)
        plt.axvspan(0,5,-5,5,facecolor='green', alpha=0.2)
    
    #draw current weight vector
    plt.quiver(0, 0, cWeight[1], cWeight[2], scale=25,label='Weight vector')
    
    
    #Delete multiple labels for w1 and w-1
    handles, labels = plt.gca().get_legend_handles_labels()
    
    #Trying to append labels for fills, but only getting a bunch of errors
    
    #handles.append(h1)
    #labels.append(r'$w^Tx \geq 0$')
    #labels.append("Test")
    #handles.append(h2)
    #labels.append(r'$w^Tx \le 0$')
    #labels.append('Test1')
    
    by_label = OrderedDict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc=3)
    
    plt.show()
    return

interact(plotStep, step=widgets.IntSlider(min=0,max=49,step=1,value=0, description='Step: '));

interactive(children=(IntSlider(value=0, description='Step: ', max=49), Output()), _dom_classes=('widget-inter…